In [27]:
import numpy as np
import tensorflow as tf
import time

In [37]:
N = 500 * 500

In [38]:
x = np.array(np.random.randn(N), dtype=np.float32)

## pytorch ops

In [39]:
import torch

In [40]:
%%timeit
Xt = torch.tensor(x).cuda()
torch.cumsum(Xt, 0)
torch.cuda.synchronize()

308 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [41]:
%%timeit
Xt = torch.tensor(x).cuda()
torch.sort(Xt, 0)
torch.cuda.synchronize()

1.26 ms ± 4.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Tensorflow ops

In [42]:
X = tf.placeholder(tf.float32, shape=(None,), name=None)
Y = tf.cumsum(X)
Z = tf.nn.top_k(X, tf.shape(X)[0])

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

sess = tf.Session(config=config)

In [43]:
%%timeit
sess.run(Y, feed_dict={X: x})

63.9 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit
sess.run(Z, feed_dict={X: x})

2.89 ms ± 49.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
